In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/Viterbi/

In [2]:
import numpy as np
from scipy.stats import norm
import scipy.linalg as la

In [3]:
observations = []
N = 0
states = []
transition_matrix = []
gaussian_means = []
gaussian_standard_deviations = []
emission_matrix = []
initial_probabilities = []
pi_vector = []
transition_matrix_changed = []
emission_matrix_changed = []
pi_vector2 = []

In [4]:
filename = 'Sample input and output for HMM/Input/data.txt'
with open(filename) as file:
    for readline in file:
        observations.append(float(readline.strip()))

In [5]:
print(observations)

[104.524317662043, 98.295390941985, 101.201300572309, 96.8258824775576, 99.7777844797209, 100.535882059444, 150.219247991425, 153.299171430611, 152.030452379689, 149.271446628265, 148.965391188617, 153.209523402459, 148.536116284592, 151.04160094104, 152.806933075742, 154.146132372137, 101.474895066967, 99.4181522342549, 99.7491679934576, 97.3801697598009, 150.700271545697, 146.088547777193, 152.142228905801, 104.346057541084, 152.941065491139, 147.6310565928, 147.400779027143, 145.465667366449, 152.762344617, 147.584883199481, 147.463428757649, 150.19276707899, 103.72260951414, 99.5057200177867, 154.25160489277, 148.616718935355, 149.20219245808, 152.519345863771, 145.867576505545, 153.743351614724, 103.925588352528, 99.9158517746542, 99.485572413414, 146.394671210885, 146.839092494125, 99.848607563626, 103.094508508026, 105.018453282783, 94.6266294243679, 100.410789870313, 100.780632342166, 98.3585134358939, 151.29390317797, 100.978651416809, 151.300787117846, 151.236439784471, 149.2

In [6]:
filename = 'Sample input and output for HMM/Input/parameters.txt'
with open(filename) as file:
    N = int(file.readline())

    for i in range(N):
        states.append(i)
    # print(states)

    for i in range(N):
        line = file.readline()
        one_row = []
        for element in line.split():
            one_row.append(float(element.strip()))
        
        #print(one_row)
        transition_matrix.append(one_row)
        #print(transition_matrix)

    line = file.readline()
    for element in line.split():
        gaussian_means.append(float(element.strip()))

    line = file.readline()
    for element in line.split():
        gaussian_standard_deviations.append(float(element.strip()))

# print(gaussian_means)
# print(gaussian_standard_deviations)

In [7]:
def gaussian_distribution(x, mean, standard_deviation):
    return norm.pdf(x, loc=mean, scale=np.sqrt(standard_deviation))

def gaussian_distribution2(x, mean, standard_deviation):
    return norm.pdf(x, loc=mean, scale=standard_deviation)

In [8]:
for state in states:
    one_row_again = []
    for observation in observations:
        probability = gaussian_distribution(observation , gaussian_means[state] , gaussian_standard_deviations[state])
        one_row_again.append(probability)
    
    emission_matrix.append(one_row_again)

print(emission_matrix)

[[1.4389397118894387e-199, 3.072909260631084e-226, 1.3788410091662133e-213, 8.909955625552642e-233, 9.716885927632503e-220, 1.882519656562696e-216, 1.946018534268589e-55, 5.518295484884176e-49, 1.360338707170505e-51, 1.661652952597701e-57, 3.50131806431804e-58, 3.629168162770999e-49, 3.879565075058746e-59, 1.1281023989469628e-53, 5.472553267992535e-50, 2.779884748139406e-47, 2.050315188414016e-212, 2.6265454703470626e-221, 7.293791295406972e-220, 2.672093860700005e-230, 2.1089762888140647e-54, 9.733425855969974e-65, 2.3240142200592563e-51, 2.6194179128639604e-200, 1.0297052760547837e-49, 3.533001926306316e-61, 1.0550173848921899e-61, 3.3224859025320027e-66, 4.433632678947177e-50, 2.773849682233438e-61, 1.4665221272596085e-61, 1.7056162332860192e-55, 6.604471104514599e-203, 6.334804891564642e-221, 4.506341138052476e-47, 5.872090824215732e-59, 1.1691262414817149e-57, 1.4026982593165182e-50, 2.950091821383754e-65, 4.349136814925479e-48, 4.652147936156678e-202, 3.873149445734235e-219, 5.17

In [9]:
#initial_probabilities = np.linalg.eig(transition_matrix)[0]
#print(transition_matrix)
#print(initial_probabilities)
#eigen_values, eigen_vectors = np.linalg.eig(transition_matrix.T)
eigen_values, eigen_vectors = la.eig(transition_matrix , left=True , right=False)
initial_probabilities = eigen_vectors[: , np.isclose(eigen_values , 1)]
initial_probabilities = initial_probabilities[:,0]
pi_vector = initial_probabilities / initial_probabilities.sum()

print(pi_vector)

[0.25 0.75]


In [10]:
def viterbi(observations , states, pi_vector , transition_matrix, emission_matrix):
    
    length = len(observations)

    probability_table = np.array([[float(0)] * length] * N)
    previous_states_table = np.array([[-1] * length] * N)

    
    for state in states:
        #probability_table[state][0] = np.log(pi_vector[state] * gaussian_distribution(observations[0], gaussian_means[state], gaussian_standard_deviations[state]))
        probability_table[state][0] = np.log(pi_vector[state] * emission_matrix[state][0])

    # print(probability_table)

    for i in range(1 , length):
        
        for current_state in states:
            maximum_probability = float('-inf')
            previous_state = -1

            for previous in states:
                counted_probability = probability_table[previous][i-1] + np.log(transition_matrix[previous][current_state])
                
                if counted_probability > maximum_probability:
                    maximum_probability = counted_probability
                    previous_state = previous

            
            maximum_probability = maximum_probability + np.log(emission_matrix[current_state][i])

            probability_table[current_state][i] = maximum_probability
            previous_states_table[current_state][i] = previous_state

    # print(probability_table)
    # print(previous_states_table)
    

    maximum_last_column = float('-inf')
    highest = -1

    predicted_seuquence = []
    prev = -1

    for i in range(N):
        if probability_table[i][length-1] > maximum_last_column:
            maximum_last_column = probability_table[i][length-1]
            highest = i

    predicted_seuquence.append(highest)
    prev = highest

    for i in range(length-2 , -1 , -1):
        predicted_seuquence.insert(0, previous_states_table[prev][i+1])
        prev = previous_states_table[prev][i+1]

    return predicted_seuquence


sequence = viterbi(observations, states, pi_vector, transition_matrix, emission_matrix)

one, zero = 0, 0
for s in sequence:
    if(s == 0):
        #print('"El Nino"')
        one += 1
    else:
        #print('"La Nina"')
        zero += 1

print(one, zero)
print(len(sequence))

291 709
1000


In [11]:
coefficients = np.array([0.0]*len(observations))
#coefficients.shape

def forward_phase(transition_matrix , pi_vector):
    length = len(observations)
    alpha_matrix = np.array([[float(0)]*length]*N)

    for i in range(N):
        alpha_matrix[i][0] = pi_vector[i] * emission_matrix[i][0]
        # print(pi_vector[i])
        # print(emission_matrix[i][0])
        # print(alpha_matrix[i][0])

    coefficients[0] = 1.0 / np.sum(alpha_matrix[: , 0])
    # print(coefficients[0])
    
    for i in range(N):
        alpha_matrix[i][0] = alpha_matrix[i][0] * coefficients[0]
        # print(alpha_matrix[i][0])
            
    for j in range(1 , length):
        for i in range(N):
            for k in range(N):
                one_state = alpha_matrix[k][j-1] * transition_matrix[k][i] * emission_matrix[i][j]
                alpha_matrix[i][j] = alpha_matrix[i][j] + one_state

        coefficients[j] = 1.0 / np.sum(alpha_matrix[: , j])
        
        for i in range(N):
            alpha_matrix[i][j] = alpha_matrix[i][j] * coefficients[j]
    
    return alpha_matrix


def backward_phase(transition_matrix):
    length = len(observations)
    beta_matrix = np.array([[float(0)]*length]*N)

    for i in range(N):
        beta_matrix[i][length-1] = 1.0

    for j in range(length-2, -1, -1):
        for i in range(N):
            for k in range(N):
                one_state = beta_matrix[k][j+1] * transition_matrix[i][k] * emission_matrix[k][j+1]
                beta_matrix[i][j] = beta_matrix[i][j] + one_state
            
            beta_matrix[i][j] = beta_matrix[i][j] * coefficients[j]
    
    return beta_matrix


# alpha_incoming = forward_phase()
# beta_incoming = backward_phase()

# print(alpha_incoming)
# print(alpha_incoming[0][0])
# print(alpha_incoming[1][0])
# print(beta_incoming)
# print(coefficients)

for i in range(N):
  one_row = []
  for j in range(N):
    one_row.append(transition_matrix[i][j])
  transition_matrix_changed.append(one_row)

# print(transition_matrix_changed)

for i in range(N):
  one_row = []
  for j in range(len(observations)):
    one_row.append(emission_matrix[i][j])
  emission_matrix_changed.append(one_row)

# print(emission_matrix_changed)

def update_phase(alpha_incoming , beta_incoming):
  length  = len(observations)

  for i in range(N):
    for j in range(N):
      upper_part = 0.0
      lower_part = 0.0

      for t in range(0 , length-1):
        # print(upper_part , "/" , lower_part)
        # all_multiplied = alpha_incoming[i][t] * beta_incoming[j][t+1] * transition_matrix_changed[i][j] * emission_matrix_changed[j][t+1]
        all_multiplied = np.log(alpha_incoming[i][t]) + np.log(beta_incoming[j][t+1]) + np.log(transition_matrix_changed[i][j]) + np.log(emission_matrix_changed[j][t+1])
        # print(all_multiplied)
        # print(alpha_incoming[i][t])
        # print(transition_matrix_changed[i][j])
        # print(emission_matrix_changed[j][t+1])
        upper_part = upper_part + np.exp(all_multiplied)
        # print(all_multiplied , "-->" , np.exp(all_multiplied))

        # all_calculated = alpha_incoming[i][t] * (beta_incoming[i][t] / coefficients[t])
        all_calculated = np.log(alpha_incoming[i][t]) + np.log(beta_incoming[i][t]) - np.log(coefficients[t])
        lower_part = lower_part + np.exp(all_calculated)

      # print(upper_part , "/" , lower_part)
      transition_matrix_changed[i][j] = upper_part / lower_part

# for i in range(20):
# update_phase()
# transition_matrix = transition_matrix_changed
# alpha_incoming = forward_phase()
# beta_incoming = backward_phase()
for i in range(20):
  eigen_values, eigen_vectors = la.eig(transition_matrix_changed , left=True , right=False)
  initial_probabilities = eigen_vectors[: , np.isclose(eigen_values , 1)]
  initial_probabilities = initial_probabilities[:,0]
  pi_vector2 = initial_probabilities / initial_probabilities.sum()
  # print(pi_vector2)
  alpha_incoming = forward_phase(transition_matrix_changed , pi_vector2)
  beta_incoming = backward_phase(transition_matrix_changed)
  update_phase(alpha_incoming , beta_incoming)
  # print(transition_matrix_changed)
# print(transition_matrix_changed)

# en_values, eigen_vectors = la.eig(transition_matrix_changed , left=True , right=False)
# initial_probabilities = eigen_vectors[: , np.isclose(eigen_values , 1)]
# initial_probabilities = initial_probabilities[:,0]
# pi_vector2 = initial_probabilities / initial_probabilities.sum()
# print(pi_vector2)


sequence = viterbi(observations, states, pi_vector2, transition_matrix_changed, emission_matrix)

one, zero = 0, 0
for s in sequence:
    if(s == 0):
        #print('"El Nino"')
        one += 1
    else:
        #print('"La Nina"')
        zero += 1

print(one, zero)
print(len(sequence))


295 705
1000


In [12]:
# norm.pdf(98 , loc=100 , scale=12)
# Q = np.array([
#     [.95, .05, 0., 0.],
#     [0., 0.9, 0.09, 0.01],
#     [0., 0.05, 0.9, 0.05],
#     [0.8, 0., 0.05, 0.15]])

# #evals, evecs = np.linalg.eig(Q.T)
# evals, evecs = la.eig(Q , left =True , right=False)
# print(evals)
# print(evecs)
# print(np.isclose(evals, 1))
# evec1 = evecs[:,np.isclose(evals, 1)]
# #evec2 = evecs[: , 0]
# print(evec1)
# #print(evec2)

# evec1 = evec1[:,0]
# print(evec1)

# stationary = evec1 / evec1.sum()
# print(stationary)

# alpha_matrix = np.array([[1.0]*10]*2)
# x = 1.0 / np.sum(alpha_matrix[: , 0])
# x

x = np.exp(np.log(10))
x

10.000000000000002